# Final

The final project will consist of a comparison between several CNN architectures for tumor detection. The goal is both to create a high-performing algorithm for differentiating kidneys with tumor from those that are normal, as well as to analyze performance across several different architecture permutations. In total, three different network designs will be tested. As each model is built and trained, ensure to serialize the final model `*.hdf5` file before moving to the next iteration.

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model files for all three models
* final compiled `*.csv` file with performance statistics across the different architectures
* final 1-page write-up with methods and results of experiments

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [1]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Imports

Use the following lines to import any additional needed libraries:

In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of kidney tumor CT exams derived from the Kidney Tumor Segmentation Challenge (KiTS). More information about the KiTS Challenge can be found here: https://kits21.kits-challenge.org/. The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/ct_kits`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [3]:
# --- Download dataset
datasets.download(name='ct/kits')

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

Since the algorithms below may require slightly different model inputs, the required generators and inputs will be defined dyanically in the code blocks later in this notebook.

### Data Generator

To accomodate these various permutations, consider the following custom code to implement a nested generator strategy:

In [4]:
def G(gen, dims=2, task='cls', binarize=True):
    """
    Custom generator to modify raw labels for 2D/3D classification or segmentation tasks
    
    :params
    
      (generator) gen      : original unmodified generator
      (int)       dims     : 2D or 3D model
      (str)       task     : 'cls' or 'seg' 
      (bool)      binarize : whether or not to binarize original 3-class labels
    
    """
    assert task in ['cls', 'seg']

    for xs, _ in gen:

        # --- Convert segmentation into classification labels
        if task == 'cls':
            axis = (2, 3, 4) if dims == 2 else (1, 2, 3, 4)
            xs['lbl'] = np.max(xs['lbl'], axis=axis, keepdims=True)
            
        # --- Binarize
        if binarize:
            xs['lbl'] = xs['lbl'] == 2

        yield xs

# Training

A total of three different network architectures will be tested. The goal is to compare the incremental benefit of several design choices. After building and training each model to convergence, do not forget to save each model as a separate `*.hdf5` file.

## 1. Classification

The first task is to create any classification model for binary tumor detection. A 2D model will predict tumor vs. no tumor on a slice-by-slice basis whereas a 3D model will predict tumor vs. no tumor on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. For those that choose a 2D model, a reduction strategy must be implemented (see details further below).

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [5]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [6]:
def create_blocks(dims=2):
    
    kernel_size = (1, 3, 3) if dims == 2 else (3, 3, 3)
    strides = (1, 2, 2) if dims == 2 else (2, 2, 2)
    
    # --- Define kwargs
    kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block components
    conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
    tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.ReLU()(x)

    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=strides)))
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=strides)))
 
    concat = lambda a, b : layers.Concatenate()([a, b])
                                     
    return conv1, conv2, tran2, concat

In [7]:
# --- Create backbone model
conv1, conv2 = create_blocks(dims=2)[:2]
# Define model input
x = Input(shape=(None, 96, 96, 1), dtype='float32')
# Define layers
l1 = conv1(8, x)
sqz = layers.AveragePooling3D((1, l1.shape[2], l1.shape[3]))(l1)
cha = int(l1.shape[-1]/2)
exc = layers.Conv3D(filters=cha, kernel_size=1, activation='relu')(sqz)
sca = layers.Conv3D(filters=l1.shape[-1], kernel_size=1, activation='sigmoid')(exc)

l1 = l1* sca
l2 = conv1 (16, conv2(16, l1))
l3 = conv1 (32, conv2(32, l2))
l4 = conv1 (48, conv2(48, l3))
l5 = conv1 (64, conv2(64, l4))
l6 = conv1 (80, conv2(80, l5))

# Reshape
n0, n1, c = l6.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l6)

# Define logits
logits = layers.Conv3D(filters=2, kernel_size=1)(f0)

# Create Model
backbone = Model(inputs = x, outputs=logits)

<KerasTensor: shape=(None, None, 96, 96, 8) dtype=float32 (created by layer 'tf.math.multiply_2')>

In [8]:
# --- Create training model
inputs = {
    'dat': Input(shape=(1, 96, 96, 1), name = 'dat'),
    'lbl': Input(shape=(1, 1, 1, 1), name = 'lbl')}
  
logits = backbone(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits=True)(
    y_true=inputs['lbl'],
    y_pred=logits)

acc = metrics.sparse_categorical_accuracy(
    y_true=inputs['lbl'], 
    y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'sec': sce, 'acc': acc})

### Compile and train model

In [9]:
# --- Compile model
training.add_loss(sce)
training.add_metric(acc, name = 'acc')
optimizer = optimizers.Adam(learning_rate=2e-4)
training.compile(optimizer=optimizer)

# Load data into memory
client.load_data_in_memory()

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='cls'),
    validation_data=G(gen_valid, dims=2, task='cls'),
    steps_per_epoch = 200,
    epochs = 10,
    validation_steps=200,
    validation_freq=5)

[ 2022-06-09 09:39:10 ] [====================] 100.000% : Iterating | 000402    Epoch 1/10
200/200 [==============================] - 18s 23ms/step - loss: 0.6564 - acc: 0.6313
Epoch 2/10
200/200 [==============================] - 5s 24ms/step - loss: 0.5522 - acc: 0.7275
Epoch 3/10
200/200 [==============================] - 5s 23ms/step - loss: 0.4761 - acc: 0.7719
Epoch 4/10
200/200 [==============================] - 5s 23ms/step - loss: 0.4199 - acc: 0.8016
Epoch 5/10
200/200 [==============================] - 8s 42ms/step - loss: 0.3449 - acc: 0.8525 - val_loss: 0.7323 - val_acc: 0.6963
Epoch 6/10
200/200 [==============================] - 5s 24ms/step - loss: 0.3143 - acc: 0.8644
Epoch 7/10
200/200 [==============================] - 5s 24ms/step - loss: 0.2606 - acc: 0.8903
Epoch 8/10
200/200 [==============================] - 5s 25ms/step - loss: 0.2391 - acc: 0.9075
Epoch 9/10
200/200 [==============================] - 5s 23ms/step - loss: 0.2046 - acc: 0.9191
Epoch 10/10
200/20

## 2. Segmentation

The second task is to create any segmentation model for binary tumor localization. A 2D model will predict tumor segmentation masks on a slice-by-slice basis whereas a 3D model will predict tumor segmentation masks on a volume basis. Regardless of implementation choice, all statistical analysis will be performed on a **volume basis**. To do so, a reduction strategy must be implemented (see details further below).

### Create generators

Use the following code cells to choose either a 2D or 3D input. As needed, feel free to modify the batch size and/or implement stratified sampling.

**2D dataset**: To select the 2D data of input size `(1, 96, 96, 1)` use the keyword `2d`:

In [ ]:
# --- Prepare generators
configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

**3D dataset**: To select the 3D data of input size `(96, 96, 96, 1)` use the keyword `3d`:

In [29]:
# --- Prepare generators
configs = {'batch': {'size': 2, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d', configs=configs, custom_layers=True)

### Define model

In [30]:
def create_blocks(dims=3):
    
    kernel_size = (1, 3, 3) if dims == 2 else (3, 3, 3)
    strides = (1, 2, 2) if dims == 2 else (2, 2, 2)
    
    # --- Define kwargs
    kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block components
    conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
    tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.ReLU()(x)

    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=strides)))
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=strides)))
 
    concat = lambda a, b : layers.Concatenate()([a, b])
                                     
    return conv1, conv2, tran2, concat, kwargs

In [31]:
# --- Create backbone model
conv1, conv2, tran2, concat, kwargs = create_blocks(dims=3)[:5]

# Define input
x = Input(shape=(96, 96, 96, 1), dtype = 'float32')

# Define Contracting layers
l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))


# Define expanding layers
l6 = tran2(48, l5)
l7 = tran2(32, conv1(48, concat(l4, l6)))
l8 = tran2(16, conv1(32, concat(l3, l7)))
l9 = conv1(8, tran2(8, conv1(16, concat(l2, l8))))


# Create logits
logits = {
    'c0': layers.Conv3D(filters=2, **kwargs)(l9),
    'c1': layers.Conv3D(filters=2, **kwargs)(l8),
    'c2': layers.Conv3D(filters=2, **kwargs)(l7),
    'c3': layers.Conv3D(filters=2, **kwargs)(l6),
}

backbone_segmentation = Model(inputs=x, outputs=logits)

In [32]:
logits

{'c0': <KerasTensor: shape=(None, 96, 96, 96, 2) dtype=float32 (created by layer 'conv3d_66')>,
 'c1': <KerasTensor: shape=(None, 48, 48, 48, 2) dtype=float32 (created by layer 'conv3d_67')>,
 'c2': <KerasTensor: shape=(None, 24, 24, 24, 2) dtype=float32 (created by layer 'conv3d_68')>,
 'c3': <KerasTensor: shape=(None, 12, 12, 12, 2) dtype=float32 (created by layer 'conv3d_69')>}

In [33]:
# --- Create training model
# Define Inputs
inputs = {
    'dat' : Input(shape=(96, 96, 96, 1), name='dat'),
    'lbl' : Input(shape=(96, 96, 96, 1), name='lbl')}

# Define first step of new wrapper model
logits = backbone_segmentation(inputs['dat'])

loss = {}
true = inputs['lbl']

for c in sorted(logits.keys()):
    
    if c != 'c0':
        true = layers.MaxPooling3D(pool_size=(2, 2, 2))(true)
    
    #Create loss for different resolution
    loss[c] = losses.SparseCategoricalCrossentropy(from_logits=True, name='sce-' + c)(
        y_true=true,
        y_pred=logits[c])
    
# Dice Score
def calculate_dsc(y_true, y_pred, weights=None, c=1):
    """
    Method to calculate the Dice score coefficient for given class

    :params

      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on

    """    
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 

    if weights is not None:
        true = true & (weights[..., 0] != 0)
        pred = pred & (weights[..., 0] != 0)

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)

    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))
  
dsc = calculate_dsc(y_true=inputs['lbl'], y_pred=logits['c0'])
training = Model(inputs=inputs, outputs={**logits, **loss, **{'dsc': dsc}})

# Add lose
for l in loss.values():
    training.add_loss(l)

# Add metric
training.add_metric(dsc, name='dsc')

In [34]:
# --- Compile model
optimizer = optimizers.Adam(learning_rate=2e-4)
training.compile(optimizer=optimizer)

# Load data into memory for faster training
client.load_data_in_memory()
# --- Train the model
training.fit(
    x=G(gen_train, dims=3, task='seg'),
    validation_data=G(gen_valid, dims=3, task='seg'),
    steps_per_epoch = 200,
    epochs = 5,
    validation_steps = 30,
    validation_freq = 5
    )

[ 2022-06-09 08:03:53 ] [====================] 100.000% : Iterating | 000402    Epoch 1/5
200/200 [==============================] - 67s 258ms/step - loss: 0.5157 - dsc: 0.0010
Epoch 2/5
200/200 [==============================] - 52s 262ms/step - loss: 0.1760 - dsc: 4.9149e-05
Epoch 3/5
200/200 [==============================] - 52s 260ms/step - loss: 0.1364 - dsc: 0.0011
Epoch 4/5
200/200 [==============================] - 52s 261ms/step - loss: 0.1138 - dsc: 0.0323
Epoch 5/5
200/200 [==============================] - 54s 272ms/step - loss: 0.0815 - dsc: 0.0325 - val_loss: 0.1983 - val_dsc: 0.0028


## 3. Custom architecture

Finally, using any of the customizations described in class, find a top-performing model that may potentially yield some incremental benefit over the two baseline models above.

### Create generators

In [4]:
# --- Choose input (may copy the generator code from above)
def G(gen, dims=2, task='seg', binarize=True):
    """
    Custom generator to modify raw labels for 2D/3D classification or segmentation tasks
    
    :params
    
      (generator) gen      : original unmodified generator
      (int)       dims     : 2D or 3D model
      (str)       task     : 'cls' or 'seg' 
      (bool)      binarize : whether or not to binarize original 3-class labels
    
    """
    assert task in ['cls', 'seg']

    for xs, _ in gen:

        # --- Convert segmentation into classification labels
        if task == 'cls':
            axis = (2, 3, 4) if dims == 2 else (1, 2, 3, 4)
            xs['lbl'] = np.max(xs['lbl'], axis=axis, keepdims=True)
            
        # --- Binarize
        if binarize:
            xs['lbl'] = xs['lbl'] == 2

        yield xs

configs = {'batch': {'size': 16, 'fold': 0}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='2d', configs=configs, custom_layers=True)

### Define model

In [5]:
def create_blocks(dims=2):
    
    kernel_size = (1, 3, 3) if dims == 2 else (3, 3, 3)
    strides = (1, 2, 2) if dims == 2 else (2, 2, 2)
    
    # --- Define kwargs
    kwargs = {
        'kernel_size': kernel_size,
        'padding': 'same',
        'kernel_initializer': 'he_normal'}

    # --- Define block components
    conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
    tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

    norm = lambda x : layers.BatchNormalization()(x)
    relu = lambda x : layers.ReLU()(x)

    conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
    conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=strides)))
    tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=strides)))
 
    concat = lambda a, b : layers.Concatenate()([a, b])
                                     
    return conv1, conv2, tran2, concat, kwargs

In [6]:
# --- Create backbone model
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))
concat = lambda a, b : layers.Concatenate()([a, b])

x = Input(shape=(None, 96, 96, 1), dtype='float32')

#Define contracting layers
l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

#Define expanding layers
l6 = tran2(48, l5)
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)

# Create logits
logits = layers.Conv3D(filters=2, **kwargs)(l10)

backbone_custmoized = Model(inputs=x, outputs=logits)

In [13]:
# --- Create training model

inputs = {
    'dat': Input(shape=(None, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(None, 96, 96, 1), name='lbl')}

logits = backbone_custmoized(inputs['dat'])

sce = losses.SparseCategoricalCrossentropy(from_logits=True)
loss = sce(y_true=inputs['lbl'], y_pred=logits)

def calculate_dsc(y_true, y_pred, c=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on
    
    """    
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)
    
    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))

dsc = calculate_dsc(y_true=inputs['lbl'], y_pred=logits)

training = Model(inputs=inputs, outputs={'logits': logits, 'loss': loss, 'dsc': dsc})
# Add loss
training.add_loss(loss)

### Compile and train model

In [14]:
# --- Compile model

# Add Metric
training.add_metric(dsc, name='dsc')
# Define an optimizer
optimizer = optimizers.Adam(learning_rate=2e-4)
training.compile(optimizer=optimizer)

# Load data into memory for faster training
client.load_data_in_memory()

# --- Train the model
training.fit(
    x=G(gen_train, dims=2, task='seg'), 
    steps_per_epoch=200, 
    epochs=10,
    validation_data=G(gen_train, dims=2, task='seg'),
    validation_steps=200,
    validation_freq=11)

[ 2022-06-09 11:53:14 ] [====================] 100.000% : Iterating | 000402    Epoch 1/10
200/200 [==============================] - 11s 37ms/step - loss: 0.0181 - dsc: 0.8848
Epoch 2/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0162 - dsc: 0.8771
Epoch 3/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0156 - dsc: 0.8855
Epoch 4/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0148 - dsc: 0.8976
Epoch 5/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0149 - dsc: 0.9000
Epoch 6/10
200/200 [==============================] - 5s 27ms/step - loss: 0.0141 - dsc: 0.9107
Epoch 7/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0138 - dsc: 0.9042
Epoch 8/10
200/200 [==============================] - 5s 25ms/step - loss: 0.0136 - dsc: 0.9109
Epoch 9/10
200/200 [==============================] - 5s 26ms/step - loss: 0.0123 - dsc: 0.9150
Epoch 10/10
200/200 [==============================] - 

# Evaluation

For each of the three models, the following metrics should be calculated for **both the training and validation** cohorts:

* accuracy
* sensitivity
* specificity
* positive predictive value (PPV)
* negative predictive value (NPV)

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please implement a prediction reduction strategy as needed for your models.

In [16]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train = G(test_train, dims=2, task='cls')
test_valid = G(test_valid, dims=2, task='cls')

train_preds = []
train_trues = []
preds = []
trues = []

for x in test_valid:
    
    # --- Aggregate preds
    pred = backbone_custmoized.predict(x['dat'])
    preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    trues.append(x['lbl'].any())

for x in test_train:

    # --- Aggregate preds
    train_pred = backbone_custmoized.predict(x['dat'])
    train_preds.append(np.argmax(pred, axis=-1).sum())

    # --- Aggregate trues
    train_trues.append(x['lbl'].any())

# --- Create Numpy arrays
preds = np.array(preds)
trues = np.array(trues)
train_preds = np.array(train_preds)
train_trues = np.array(train_trues)


[ 2022-06-09 11:55:10 ] [====================] 100.000% : Iterating | 000321    

In [17]:
# --- Apply threshold
thresh = np.median(preds)
preds_ = preds >= thresh

In [18]:
# --- Calculate TP/TN/FN/FP
corr = preds_ == trues
tp = np.sum(corr & trues)
tn = np.sum(corr & ~trues)
fn = np.sum(~corr & trues)
fp = np.sum(~corr & ~trues)

# --- Calculate stats
acc = (tp + tn) / corr.size
sen = tp / (tp + fn)
spe = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

Acc: 0.6790
Sen: 0.6596
Spe: 0.7059
PPV: 0.7561
NPV: 0.6000


In [19]:
# --- Apply threshold
train_thresh = np.median(train_preds)
train_preds_ = train_preds >= train_thresh

In [20]:
# --- Calculate TP/TN/FN/FP
corr = train_preds_ == train_trues
tp = np.sum(corr & train_trues)
tn = np.sum(corr & ~train_trues)
fn = np.sum(~corr & train_trues)
fp = np.sum(~corr & ~train_trues)

# --- Calculate stats
train_acc = (tp + tn) / corr.size
train_sen = tp / (tp + fn)
train_spe = tn / (tn + fp)
train_ppv = tp / (tp + fp)
train_npv = tn / (tn + fn)

print('Acc: {:0.4f}'.format(acc))
print('Sen: {:0.4f}'.format(sen))
print('Spe: {:0.4f}'.format(spe))
print('PPV: {:0.4f}'.format(ppv))
print('NPV: {:0.4f}'.format(npv))

Acc: 0.6790
Sen: 0.6596
Spe: 0.7059
PPV: 0.7561
NPV: 0.6000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in long_scalars
  del sys.path[0]


### Performance

The following minimum **validation cohort** performance metrics must be met for full credit:

1. **Classification**: accuracy > 0.55
2. **Segmentation**: accuracy > 0.55
3. **Custom architecture**: accuracy > 0.60

**Bonus**: the top three overall models based on **validation cohort** accuracy will recieve a +5 point (+15%) extra credit towards the final assignment.

### Results

When ready, create a `*.csv` file with your compiled **training and validation** cohort statistics for the three different models. Consider the following table format (although any format that contains the required information is sufficient):

```
          TRAINING                              VALIDATION
          accuracy | sens | spec | PPV |  NPV | accuracy | sens | spec | PPV |  NPV
model 1
model 2
model 3
```

As above, statistics for both training and validation should be provided.

In [21]:
# --- Create *.csv
df = pd.DataFrame(index=np.arange(1))
df['accuracy'] = acc
df['sens'] = sen
df['spec'] = spe
df['PPV'] = ppv
df['NPV'] = npv
df['train_accuracy'] = train_acc
df['train_sens'] = train_sen
df['train_spec'] = train_spe
df['train_PPV'] = train_ppv
df['train_NPV'] = train_npv

# --- Serialize *.csv
df.to_csv('./model3_results.csv')

In [23]:
backbone_custmoized.save('./model_cus.hdf5')

# Summary

In addition to algorithm training as above, a 1-2 page write-up is required for this project. The goal is to *briefly* summarize algorithm design and key results. The write-up should be divided into three sections: methods; results; discussion. More detailed information and tips can be found here: https://github.com/peterchang77/dl_tutor/blob/master/cs190/spring_2021/notebooks/midterm/checklist.md.

### Methods

In this section, include details such as:

* **Data**: How much data was used. How many cases were utilized for training and validation?
* **Network design**: What are the different network architectures? How many layers and parameters? Were 2D or 3D operations used? Recall that the `model.summary(...)` can be used to provide key summary statistics for this purpose. If desired, feel free to include a model figure or diagram.
* **Implementation**: How was training implemented. What are the key hyperparameters (e.g. learning rate, batch size, optimizer, etc)? How many training iterations were required for convergence? Did these hyperparameters change during the course of training?
* **Statistics**: What statistics do you plan to use to evaluate model accuracy? 

### Results

In this section, briefly summarize experimental results (a few sentences), and include the result table(s) as derived above.

### Discussion

Were the results expected or unexpected? What accounts for the differences in performance between the algorithms? How did you choose the network architecture implemented in your final model? Feel free to elaborate on any additional observations noted during the course of this expierment.

# Submission


### Canvas

Once you have completed the midterm assignment, download the necessary files from Google Colab and your Google Drive. As in prior assigments, be sure to prepare:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv` (compiled for all three parts)
* final (trained) model: `[UCInetID]_model.hdf5` (three separate files for all three parts)

In addition, submit the summary write-up as in any common document format (`.docx`, `.tex`, `.pdf`, etc):

* final summary write-up: `[UCInetID]_summary.[docx|tex|pdf]`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadsheet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.